<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: Юрий Исаков и Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Тема 4. Линейные модели классификации и регрессии
## <center>  Практика. Идентификация пользователя с помощью логистической регрессии

Тут мы воспроизведем парочку бенчмарков нашего соревнования и вдохновимся побить третий бенчмарк, а также остальных участников. Веб-формы для отправки ответов тут не будет, ориентир – [leaderboard](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard) соревнования.

In [1]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

### 1. Загрузка и преобразование данных
Зарегистрируйтесь на [Kaggle](www.kaggle.com), если вы не сделали этого раньше, зайдите на [страницу](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования и скачайте данные. Первым делом загрузим обучающую и тестовую выборки и посмотрим на данные.

In [2]:
# загрузим обучающую и тестовую выборки
train_df = pd.read_csv('../../data/train_sessions.csv', index_col='session_id')
test_df = pd.read_csv('../../data/test_sessions.csv', index_col='session_id')

# приведем колонки time1, ..., time10 к временному формату
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# отсортируем данные по времени
train_df = train_df.sort_values(by='time1')

# посмотрим на заголовок обучающей выборки
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


In [51]:
train_df['target'].value_counts()

0    251264
1      2297
Name: target, dtype: int64

In [52]:
train_df[train_df['target'] == 1].head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
251175,270,2013-02-12 16:25:10,270.0,2013-02-12 16:25:11,270.0,2013-02-12 16:32:10,21.0,2013-02-12 16:32:11,21.0,2013-02-12 16:32:24,...,2013-02-12 16:32:25,21.0,2013-02-12 16:32:25,7832.0,2013-02-12 16:32:26,30.0,2013-02-12 16:32:27,7832.0,2013-02-12 16:32:27,1
196388,29,2013-02-12 16:32:27,7832.0,2013-02-12 16:32:28,37.0,2013-02-12 16:32:29,7832.0,2013-02-12 16:32:34,7832.0,2013-02-12 16:32:35,...,2013-02-12 16:32:35,7832.0,2013-02-12 16:32:42,29.0,2013-02-12 16:32:42,7832.0,2013-02-12 16:32:51,7832.0,2013-02-12 16:32:53,1
172448,29,2013-02-12 16:32:53,7832.0,2013-02-12 16:33:11,7832.0,2013-02-12 16:33:12,29.0,2013-02-12 16:33:13,37.0,2013-02-12 16:33:15,...,2013-02-12 16:33:24,29.0,2013-02-12 16:33:24,7832.0,2013-02-12 16:33:33,29.0,2013-02-12 16:33:34,270.0,2013-02-12 16:33:46,1
70129,167,2013-02-12 16:33:50,167.0,2013-02-12 16:33:51,1515.0,2013-02-12 16:33:52,167.0,2013-02-12 16:33:52,37.0,2013-02-12 16:33:52,...,2013-02-12 16:33:52,855.0,2013-02-12 16:33:52,1515.0,2013-02-12 16:33:53,855.0,2013-02-12 16:33:53,1514.0,2013-02-12 16:33:53,1
206254,1520,2013-02-12 16:33:55,1522.0,2013-02-12 16:33:56,1522.0,2013-02-12 16:34:01,1515.0,2013-02-12 16:34:12,1515.0,2013-02-12 16:34:13,...,2013-02-12 16:34:24,1514.0,2013-02-12 16:34:24,1515.0,2013-02-12 16:34:24,1520.0,2013-02-12 16:34:25,1521.0,2013-02-12 16:34:25,1


In [3]:
x = train_df[train_df['target']==1]['site1']

In [3]:
from collections import Counter

In [5]:
x1 = Counter(x.values)

In [6]:
for k in sorted(x1, key = lambda x : x1[x], reverse=True):
    print k, x1[k]

80 142
77 141
76 130
29 98
21 80
22 56
81 54
23 51
879 47
82 43
881 40
75 35
35 32
37 31
30 30
33 30
3000 28
941 27
78 25
733 25
2078 21
52 20
855 19
270 18
229 17
39 16
335 16
942 16
1057 16
1514 16
704 15
7832 15
3 14
3559 14
617 14
38 12
265 12
12619 12
14 11
2080 11
1307 10
99 9
2077 9
616 9
27307 9
3560 9
1515 9
1519 9
1 8
32 8
5396 8
4694 8
233 7
263 7
268 7
677 7
940 7
1324 7
8 6
8316 6
143 6
167 6
272 6
397 6
27189 6
666 6
25383 6
17283 6
5392 6
5402 6
1518 6
1521 6
1524 6
2 5
148 5
269 5
2574 5
27165 5
27336 5
774 5
896 5
1345 5
1516 5
1520 5
3846 5
6088 5
11 4
2079 4
51 4
106 4
4240 4
170 4
271 4
486 4
3517 4
5397 4
5400 4
1439 4
1517 4
1523 4
2372 4
12 3
16 3
2081 3
63 3
175 3
18611 3
240 3
18876 3
448 3
515 3
27168 3
27190 3
27193 3
27221 3
27228 3
644 3
796 3
7013 3
876 3
11241 3
3150 3
3240 3
5301 3
13547 3
5398 3
5403 3
1312 3
1522 3
4693 3
5879 3
3847 3
1919 3
1995 3
24553 3
4 2
6 2
7 2
27311 2
107 2
117 2
140 2
2271 2
4369 2
291 2
27358 2
2401 2
355 2
390 2
2512 2
2736

В обучающей выборке содержатся следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - target – целевая переменная, 1 для сессий Элис, 0 для сессий других пользователей
    
Сессии пользователей выделены таким образом, что они не могут быть длиннее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд либо когда сессия заняла по времени более 30 минут.

В таблице встречаются пропущенные значения, это значит, что сессия состоит менее, чем из 10 сайтов. Заменим пропущенные значения нулями и приведем признаки к целому типу. Также загрузим словарь сайтов и посмотрим, как он выглядит:

In [4]:
# приведем колонки site1, ..., site10 к целочисленному формату и заменим пропуски нулями
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

# загрузим словарик сайтов
with open(r"../../data/site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# датафрейм словарика сайтов
sites_dict_df = pd.DataFrame(list(site_dict.keys()), 
                          index=list(site_dict.values()), 
                          columns=['site'])
print(u'всего сайтов:', sites_dict_df.shape[0])
sites_dict_df.head()

(u'\u0432\u0441\u0435\u0433\u043e \u0441\u0430\u0439\u0442\u043e\u0432:', 48371)


,site
13852,i1-js-14-3-01-11544-644949368-i.init.cedexis-r...
6240,static.vol24.fr
39840,mars.nasa.gov
3899,scholar.google.fr
9677,9zouxfza1h.s.ad6media.fr


Выделим целевую переменную и объединим выборки, чтобы вместе привести их к разреженному формату.

In [5]:
# наша целевая переменная
y_train = train_df['target']

# объединенная таблица исходных данных
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# индекс, по которому будем отделять обучающую выборку от тестовой
idx_split = train_df.shape[0]

In [9]:
idx_split

253561

Для самой первой модели будем использовать только посещенные сайты в сессии (но не будем обращать внимание на временные признаки). За таким выбором данных для модели стоит такая идея:  *у Элис есть свои излюбленные сайты, и чем чаще вы видим эти сайты в сессии, тем выше вероятность, что это сессия Элис и наоборот.*

Подготовим данные, из всей таблицы выберем только признаки `site1, site2, ... , site10`. Напомним, что пропущенные значения заменены нулем. Вот как выглядят первые строки таблицы:

In [6]:
# табличка с индексами посещенных сайтов в сессии
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


Сессии представляют собой последовательность индексов сайтов и данные в таком виде неудобны для линейных методов. В соответствии с нашей гипотезой (у Элис есть излюбленные сайты) надо преобразовать эту таблицу таким образом, чтобы каждому возможному сайту соответствовал свой отдельный признак (колонка), а его значение равнялось бы количеству посещений этого сайта в сессии. Это делается в две строчки:

In [7]:
from scipy.sparse import csr_matrix

In [2]:
csr_matrix?

In [8]:
# последовательность с индексами
sites_flatten = full_sites.values.flatten()

# искомая матрица
full_sites_sparse = csr_matrix((np.ones_like(sites_flatten), # [1] * sites_flatten.shape[0],
                                sites_flatten,
                                range(0, sites_flatten.shape[0] + 10, 10)))[:, 1:]

In [13]:
np.ones_like(sites_flatten)

array([1, 1, 1, ..., 1, 1, 1])

In [14]:
sites_flatten.shape

(3363580,)

In [15]:
full_sites_sparse.shape

(336358, 48371)

In [75]:
X_train_sparse = full_sites_sparse[:idx_split]
X_test_sparse = full_sites_sparse[idx_split:]

In [76]:
X_train_sparse.shape, y_train.shape

((253561, 48371), (253561,))

In [77]:
X_train_sparse

<253561x48371 sparse matrix of type '<type 'numpy.int32'>'
	with 2412880 stored elements in Compressed Sparse Row format>

In [78]:
X_test_sparse.shape

(82797, 48371)

Еще один плюс использования разреженных матриц в том, что для них имеются специальные реализации как матричных операций, так и алгоритмов машинного обучения, что подчас позволяет ощутимо ускорить операции за счет особенностей структуры данных. Это касается и логистической регрессии. Вот теперь у нас все готово для построения нашей первой модели.

### 2. Построение первой модели

Итак, у нас есть алгоритм и данные для него, построим нашу первую модель, воспользовавшись релизацией [логистической регрессии](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) из пакета `sklearn` с параметрами по умолчанию. Первые 90% данных будем использовать для обучения (обучающая выборка отсортирована по времени), а оставшиеся 10% для проверки качества (validation). 

**Напишите простую функцию, которая будет возвращать качество модели на отложенной выборке, и обучите наш первый классификатор**.

In [10]:
def get_auc_lr_valid(X, y, C=1.0, ratio = 0.9, seed=17):
    '''
    X, y – выборка
    ratio – в каком отношении поделить выборку
    C, seed – коэф-т регуляризации и random_state 
              логистической регрессии
    '''
    train_len = int(ratio * X.shape[0])
    X_train = X[:train_len, :]
    X_valid = X[train_len:, :]
    y_train = y[:train_len]
    y_valid = y[train_len:]
    
    logit = LogisticRegression(C=C, random_state=seed)
    logit.fit(X_train,y_train)
    
    valid_pred = logit.predict_proba(X_valid)[:, 1]
    
    return roc_auc_score(y_valid, valid_pred)


**Посмотрите, какой получился ROC AUC на отложенной выборке.**

In [11]:
get_auc_lr_valid(X_train_sparse, y_train)

0.9195265581732923

Будем считать эту модель нашей первой отправной точкой (baseline). Для построения модели для прогноза на тестовой выборке **необходимо обучить модель заново уже на всей обучающей выборке** (пока наша модель обучалась лишь на части данных), что повысит ее обобщающую способность:

In [12]:
# функция для записи прогнозов в файл
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

**Обучите модель на всей выборке, сделайте прогноз для тестовой выборки и сделайте посылку в соревновании**.

In [13]:
logit = LogisticRegression(random_state=17)
logit.fit(X_train_sparse, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [14]:
res_pred = logit.predict_proba(X_test_sparse)[:, 1]

In [26]:
res_pred.shape

(82797,)

In [27]:
write_to_submission_file(res_pred, 'res1.txt')

Если вы выполните эти действия и загрузите ответ на [странице](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования, то воспроизведете первый бенчмарк "Logit".

### 3. Улучшение модели, построение новых признаков

Создайте такой признак, который будет представлять собой число вида ГГГГММ от той даты, когда проходила сессия, например 201407 -- 2014 год и 7 месяц. Таким образом, мы будем учитывать помесячный [линейный тренд](http://people.duke.edu/~rnau/411trend.htm) за весь период предоставленных данных.

In [15]:
time = ['time%s' % i for i in range(1, 11)]

In [29]:
time

['time1',
 'time2',
 'time3',
 'time4',
 'time5',
 'time6',
 'time7',
 'time8',
 'time9',
 'time10']

In [16]:
train_df[time].head()

,time1,time2,time3,time4,time5,time6,time7,time8,time9,time10
session_id,,,,,,,,,,
21669,2013-01-12 08:05:57,2013-01-12 08:05:57,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
54843,2013-01-12 08:37:23,2013-01-12 08:37:23,2013-01-12 09:07:07,2013-01-12 09:07:09,NaT,NaT,NaT,NaT,NaT,NaT
77292,2013-01-12 08:50:13,2013-01-12 08:50:14,2013-01-12 08:50:15,2013-01-12 08:50:15,2013-01-12 08:50:16,2013-01-12 08:50:16,2013-01-12 08:50:16,2013-01-12 08:50:16,2013-01-12 08:50:17,2013-01-12 08:50:17
114021,2013-01-12 08:50:17,2013-01-12 08:50:17,2013-01-12 08:50:18,2013-01-12 08:50:18,2013-01-12 08:50:18,2013-01-12 08:50:18,2013-01-12 08:50:19,2013-01-12 08:50:19,2013-01-12 08:50:19,2013-01-12 08:50:20
146670,2013-01-12 08:50:20,2013-01-12 08:50:20,2013-01-12 08:50:20,2013-01-12 08:50:21,2013-01-12 08:50:21,2013-01-12 08:50:21,2013-01-12 08:50:21,2013-01-12 08:50:22,2013-01-12 08:50:22,2013-01-12 08:50:22


In [17]:
train_df['time1'].head()

session_id
21669    2013-01-12 08:05:57
54843    2013-01-12 08:37:23
77292    2013-01-12 08:50:13
114021   2013-01-12 08:50:17
146670   2013-01-12 08:50:20
Name: time1, dtype: datetime64[ns]

In [18]:
train_df['time1'].apply(lambda ts: int(ts.year * 100 + ts.month)).head()

session_id
21669     201301
54843     201301
77292     201301
114021    201301
146670    201301
Name: time1, dtype: int64

In [19]:
new_feat_train = pd.DataFrame(index=train_df.index)
new_feat_test = pd.DataFrame(index=test_df.index)

In [20]:
new_feat_train['year_month_start'] = train_df['time1'].apply(lambda ts: int(ts.year * 100 + ts.month))
new_feat_test['year_month_start'] = test_df['time1'].apply(lambda ts: int(ts.year * 100 + ts.month))

In [21]:
new_feat_train.head()

,year_month_start
session_id,
21669,201301
54843,201301
77292,201301
114021,201301
146670,201301


In [22]:
scaler = StandardScaler()
scaler.fit(new_feat_train['year_month_start'].values.reshape(-1, 1))

new_feat_train['year_month_start_scaler'] = scaler.transform(new_feat_train['year_month_start'].values.reshape(-1, 1))
new_feat_test['year_month_start_scaler'] = scaler.transform(new_feat_test['year_month_start'].values.reshape(-1, 1))


C:\Python27\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [36]:
new_feat_train.head()

,year_month_start,year_month_start_scaler
session_id,,
21669,201301,-1.744405
54843,201301,-1.744405
77292,201301,-1.744405
114021,201301,-1.744405
146670,201301,-1.744405


Добавьте новый признак, предварительно отмасштабировав его с помощью `StandardScaler`, и снова посчитайте ROC AUC на отложенной выборке.

In [23]:
new_feat_train['year_month_start_scaler'].values.reshape(-1, 1)

array([[-1.74440496],
       [-1.74440496],
       [-1.74440496],
       ...,
       [ 0.68162559],
       [ 0.68162559],
       [ 0.68162559]])

In [25]:
import scipy.sparse as sp

In [26]:
X_train_sparse_time = sp.hstack([X_train_sparse, new_feat_train['year_month_start_scaler'].values.reshape(-1, 1)])

In [27]:
X_train_sparse_time = X_train_sparse_time.tocsr()

In [43]:
X_train_sparse_time

<253561x48372 sparse matrix of type '<type 'numpy.float64'>'
	with 1683237 stored elements in Compressed Sparse Row format>

In [28]:
X_test_sparse_time = sp.hstack([X_test_sparse, new_feat_test['year_month_start_scaler'].values.reshape(-1, 1)])

In [29]:
X_test_sparse_time = X_test_sparse_time.tocsr()

In [46]:
X_test_sparse_time

<82797x48372 sparse matrix of type '<type 'numpy.float64'>'
	with 520019 stored elements in Compressed Sparse Row format>

In [47]:
X_train_sparse_time.shape, X_test_sparse_time.shape

((253561, 48372), (82797, 48372))

In [30]:
get_auc_lr_valid(X_train_sparse_time, y_train)

0.919692607137811

In [31]:
logit = LogisticRegression(random_state=17)
logit.fit(X_train_sparse_time, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [50]:
res_pred = logit.predict_proba(X_test_sparse_time)[:, 1]

In [117]:
write_to_submission_file(res_pred, 'res2.txt')

**Добавьте два новых признака: start_hour и morning.**

Признак `start_hour` – это час в который началась сессия (от 0 до 23), а бинарный признак `morning` равен 1, если сессия началась утром и 0, если сессия началась позже (будем считать, что утро это если `start_hour равен` 11 или меньше).

**Посчитйте ROC AUC на отложенной выборке для выборки с:**
- сайтами, `start_month` и `start_hour`
- сайтами, `start_month` и `morning`
- сайтами, `start_month`, `start_hour` и `morning`

In [32]:
new_feat_train.head()

,year_month_start,year_month_start_scaler
session_id,,
21669,201301,-1.744405
54843,201301,-1.744405
77292,201301,-1.744405
114021,201301,-1.744405
146670,201301,-1.744405


In [33]:
train_df['time1'].apply(lambda t : int(t.hour)).head()

session_id
21669     8
54843     8
77292     8
114021    8
146670    8
Name: time1, dtype: int64

In [34]:
new_feat_train['start_hour'] = train_df['time1'].apply(lambda t : int(t.hour))
new_feat_test['start_hour'] = test_df['time1'].apply(lambda t : int(t.hour))

In [64]:
new_feat_test.head()

,year_month_start,year_month_start_scaler,start_hour
session_id,,,
1,201410,0.822948,11
2,201407,0.752287,11
3,201412,0.870055,15
4,201411,0.846501,10
5,201405,0.705179,15


In [79]:
new_feat_train_morning['morning'] = new_feat_train['start_hour'].apply(time_lambda)
new_feat_test_morning['morning'] = new_feat_test['start_hour'].apply(time_lambda)

## Сайты, месяц_год, час

In [85]:
X_train_year_hour = sp.hstack([X_train_sparse, new_feat_train['year_month_start_scaler'].values.reshape(-1, 1), new_feat_train['start_hour'].values.reshape(-1, 1)])

In [89]:
X_train_year_hour = X_train_year_hour.tocsc()

In [90]:
X_test_year_hour = sp.hstack([X_test_sparse, new_feat_test['year_month_start_scaler'].values.reshape(-1, 1), new_feat_test['start_hour'].values.reshape(-1, 1)])
X_test_year_hour = X_test_year_hour.tocsc()

In [92]:
get_auc_lr_valid(X_train_year_hour, y_train)

0.9573176420080936

In [96]:
logit = LogisticRegression(random_state=17)
logit.fit(X_train_year_hour, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [97]:
pred1 = logit.predict_proba(X_test_year_hour)[:, 1]

In [98]:
write_to_submission_file(pred1, 'res_sites_year_hour.txt')

## Сайты, месяц_год, утро

In [103]:
X_train_year_morning = sp.hstack([X_train_sparse, new_feat_train['year_month_start_scaler'].values.reshape(-1, 1), new_feat_train['morning'].values.reshape(-1, 1)])
X_train_year_morning = X_train_year_morning.tocsc()

In [104]:
X_test_year_morning = sp.hstack([X_test_sparse, new_feat_test['year_month_start_scaler'].values.reshape(-1, 1), new_feat_test['morning'].values.reshape(-1, 1)])
X_test_year_morning = X_test_year_morning.tocsc()

In [105]:
get_auc_lr_valid(X_train_year_morning, y_train)

0.9486636228381934

In [107]:
logit = LogisticRegression(random_state=17)
logit.fit(X_train_year_morning, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [108]:
pred2 = logit.predict_proba(X_test_year_morning)[:,1]

In [109]:
write_to_submission_file(pred2, 'sites_year_morning.txt')

# Сайты, месяц_год, часы, утро

In [110]:
X_train_year_hour_morning = sp.hstack([X_train_sparse, new_feat_train['year_month_start_scaler'].values.reshape(-1, 1), new_feat_train['morning'].values.reshape(-1, 1), new_feat_train['start_hour'].values.reshape(-1, 1)])
X_train_year_hour_morning = X_train_year_hour_morning.tocsc()

X_test_year_hour_morning = sp.hstack([X_test_sparse, new_feat_test['year_month_start_scaler'].values.reshape(-1, 1), new_feat_test['morning'].values.reshape(-1, 1), new_feat_test['start_hour'].values.reshape(-1, 1)])
X_test_year_hour_morning = X_test_year_hour_morning.tocsc()

In [111]:
get_auc_lr_valid(X_train_year_hour_morning, y_train)

0.9584990049138418

In [112]:
logit = LogisticRegression(random_state=17)
logit.fit(X_train_year_hour_morning, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [113]:
pred3 = logit.predict_proba(X_test_year_hour_morning)[:, 1]

In [114]:
write_to_submission_file(pred3, 'sites_year_hour_morning.txt')

### 4. Подбор коэффицициента регуляризации

Итак, мы ввели признаки, которые улучшают качество нашей модели по сравнению с первым бейслайном. Можем ли мы добиться большего значения метрики? После того, как мы сформировали обучающую и тестовую выборки, почти всегда имеет смысл подобрать оптимальные гиперпараметры -- характеристики модели, которые не изменяются во время обучения. Например, на 3 неделе вы проходили решающие деревья, глубина дерева это гиперпараметр, а признак, по которому происходит ветвление и его значение -- нет. В используемой нами логистической регрессии веса каждого признака изменяются и во время обучения находится их оптимальные значения, а коэффициент регуляризации остается постоянным. Это тот гиперпараметр, который мы сейчас будем оптимизировать.

Посчитайте качество на отложенной выборке с коэффициентом регуляризации, который по умолчанию `C=1`:

In [ ]:
# Ваш код здесь

Постараемся побить этот результат за счет оптимизации коэффициента регуляризации. Возьмем набор возможных значений C и для каждого из них посчитаем значение метрики на отложенной выборке.

Найдите `C` из `np.logspace(-3, 1, 10)`, при котором ROC AUC на отложенной выборке максимален. 

In [ ]:
# Ваш код здесь

Наконец, обучите модель с найденным оптимальным значением коэффициента регуляризации и с построенными признаками `start_hour`, `start_month` и `morning`. Если вы все сделали правильно и загрузите это решение, то повторите второй бенчмарк соревнования.

In [ ]:
# Ваш код здесь

In [110]:
# Day, evening, night

In [98]:
time_lambda_night = lambda x : 1 if x < 6 else 0
time_lambda_morning = lambda x : 1 if 6 <= x <= 11 else 0
time_lambda_day = lambda x : 1 if 12 <= x <= 18 else 0
time_lambda_evening = lambda x : 1 if 19 <= x < 24 else 0

In [99]:
def apply_time_lambda(train, test, feature, time_lambda):
    train[feature] = train['start_hour'].apply(time_lambda)
    test[feature] = test['start_hour'].apply(time_lambda)

In [100]:
apply_time_lambda(new_feat_train, new_feat_test, 'morning', time_lambda_morning)
apply_time_lambda(new_feat_train, new_feat_test, 'day', time_lambda_day)
apply_time_lambda(new_feat_train, new_feat_test, 'evening', time_lambda_evening)
apply_time_lambda(new_feat_train, new_feat_test, 'night', time_lambda_night)

In [101]:
new_feat_train.head()

,year_month_start,year_month_start_scaler,start_hour,morning,day,evening,night
session_id,,,,,,,
21669,201301,-1.744405,8,1,0,0,0
54843,201301,-1.744405,8,1,0,0,0
77292,201301,-1.744405,8,1,0,0,0
114021,201301,-1.744405,8,1,0,0,0
146670,201301,-1.744405,8,1,0,0,0


In [102]:
def add_feats_to_sites(X_train_sites, X_test_sites, feature, new_feat_train, new_feat_test):
    result_train = sp.hstack([X_train_sites, new_feat_train[feature].values.reshape(-1, 1)])
    result_train = result_train.tocsc()
    
    result_test = sp.hstack([X_test_sites, new_feat_test[feature].values.reshape(-1, 1)])
    result_test = result_test.tocsc()
    
    return result_train, result_test

In [103]:
new_train_data, new_test_data = add_feats_to_sites(X_train_sparse, X_test_sparse, 'year_month_start_scaler', new_feat_train, new_feat_test)

In [87]:
#new_train_data, new_test_data = add_feats_to_sites(new_train_data, new_test_data, 'start_hour', new_feat_train, new_feat_test)

In [104]:
new_train_data, new_test_data = add_feats_to_sites(new_train_data, new_test_data, 'morning', new_feat_train, new_feat_test)

In [105]:
new_train_data, new_test_data = add_feats_to_sites(new_train_data, new_test_data, 'day', new_feat_train, new_feat_test)

In [106]:
new_train_data, new_test_data = add_feats_to_sites(new_train_data, new_test_data, 'evening', new_feat_train, new_feat_test)

In [107]:
new_train_data, new_test_data = add_feats_to_sites(new_train_data, new_test_data, 'night', new_feat_train, new_feat_test)

In [108]:
new_train_data.shape, new_test_data.shape

((253561, 48376), (82797, 48376))

In [94]:
get_auc_lr_valid(new_train_data, y_train)

0.9501488553490107

In [109]:
logit = LogisticRegression(random_state=17)
logit.fit(new_train_data, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [111]:
pred7 = logit.predict_proba(new_test_data)[:, 1]

In [112]:
write_to_submission_file(pred7, 'sites_year_hour_dayparts1.txt')

In [115]:
skf = StratifiedKFold(n_splits=5)

In [114]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold

In [116]:
gs = GridSearchCV(estimator=LogisticRegression(), param_grid={'C' : np.logspace(-3, 1, 10)}, cv=skf, scoring='roc_auc', n_jobs=-1)

In [117]:
gs.fit(new_train_data, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'C': array([1.00000e-03, 2.78256e-03, 7.74264e-03, 2.15443e-02, 5.99484e-02,
       1.66810e-01, 4.64159e-01, 1.29155e+00, 3.59381e+00, 1.00000e+01])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [120]:
gs.grid_scores_

C:\Python27\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.79786, std: 0.06277, params: {'C': 0.001},
 mean: 0.86477, std: 0.04885, params: {'C': 0.0027825594022071257},
 mean: 0.90143, std: 0.03360, params: {'C': 0.007742636826811269},
 mean: 0.91827, std: 0.02610, params: {'C': 0.021544346900318832},
 mean: 0.92644, std: 0.02112, params: {'C': 0.05994842503189409},
 mean: 0.93156, std: 0.01778, params: {'C': 0.1668100537200059},
 mean: 0.93464, std: 0.01515, params: {'C': 0.46415888336127775},
 mean: 0.93571, std: 0.01289, params: {'C': 1.2915496650148828},
 mean: 0.93526, std: 0.01085, params: {'C': 3.593813663804626},
 mean: 0.93382, std: 0.00920, params: {'C': 10.0}]

In [119]:
gs.best_score_

0.9357069088996511

In [125]:
get_auc_lr_valid(new_train_data, y_train, C=2)

0.9492315102968474

In [122]:
logit = LogisticRegression(random_state=17, C=1.5)
logit.fit(new_train_data, y_train)

LogisticRegression(C=1.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [123]:
pred8 = logit.predict_proba(new_test_data)[:, 1]

In [124]:
write_to_submission_file(pred8, 'model_1.5_simple_features.txt')